In [1]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML    

In [74]:
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option("display.max_rows", 500)

In [112]:
df = pd.read_csv('data/plays.csv')
df = df.set_index('Dataline')
df[df['Play'] == 'King Lear']

df['ActSceneLine'] = df[['ActSceneLine']].applymap(lambda x: str(x))
df['Act'] = [ np.nan if line == np.nan else line.split('.')[0] for line in df.ActSceneLine]
df['Scene'] = [ np.nan if line == np.nan else line.split('.')[1] if len(line.split('.')) > 1 else np.nan for line in df.ActSceneLine]

df = df.dropna()

df['Act'] = [int(float(a)) for a in df['Act']]
df['Scene'] = [int(float(s)) for s in df['Scene']]
df['PlayerLinenumber'] = df[['PlayerLinenumber']].applymap(lambda x : round(x))

df['Code'] = ''
for i,r in df.iterrows():
    df.at[i, 'Code'] = r['Play'] + '||||' + '.'.join([str(r['Act']), str(r['Scene']), '{:03}'.format(r['PlayerLinenumber'])]) + '||||' + r['Player']

df

,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine,Act,Scene,Code
Dataline,,,,,,,,
4,Henry IV,1,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,",1,1,Henry IV||||1.1.001||||KING HENRY IV
5,Henry IV,1,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,",1,1,Henry IV||||1.1.001||||KING HENRY IV
6,Henry IV,1,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils,1,1,Henry IV||||1.1.001||||KING HENRY IV
7,Henry IV,1,1.1.4,KING HENRY IV,To be commenced in strands afar remote.,1,1,Henry IV||||1.1.001||||KING HENRY IV
8,Henry IV,1,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil,1,1,Henry IV||||1.1.001||||KING HENRY IV
9,Henry IV,1,1.1.6,KING HENRY IV,Shall daub her lips with her own children's bl...,1,1,Henry IV||||1.1.001||||KING HENRY IV
10,Henry IV,1,1.1.7,KING HENRY IV,"Nor more shall trenching war channel her fields,",1,1,Henry IV||||1.1.001||||KING HENRY IV
11,Henry IV,1,1.1.8,KING HENRY IV,Nor bruise her flowerets with the armed hoofs,1,1,Henry IV||||1.1.001||||KING HENRY IV
12,Henry IV,1,1.1.9,KING HENRY IV,"Of hostile paces: those opposed eyes,",1,1,Henry IV||||1.1.001||||KING HENRY IV


In [130]:
m = df[['Code', 'PlayerLine']].groupby(['Code']).sum()
m = m.reset_index()

m['Play'] = [c.split('||||')[0] for c in m['Code']]
m['Address'] = [c.split('||||')[1] for c in m['Code']]
m['Act'] = [a.split('.')[0] for a in m['Address']]
m['Scene'] = [a.split('.')[1] for a in m['Address']]
m['Player'] = [c.split('||||')[2] for c in m['Code']]

m['Targ'] = m['Prev'] = m['Next'] = ''

# fill targ prev and next

for i, r in m.iterrows():
    line_number = int(r['Address'].split('.')[-1])
    
    if line_number == 1:  # the first line of each scene
        r['Prev'] = r['Targ'] = np.NaN
        r['Next'] = m.at[i + 1, 'Player']
    elif i + 1 >= len(m) or m.at[i + 1, 'Act'] != r['Act'] or m.at[i + 1, 'Scene'] != r['Scene']: # the last line of each scene, act 
        r['Next'] = r['Targ'] = np.NaN
        r['Prev'] = m.at[i - 1, 'Player']
    else: # only detect target here
        r['Prev'] = m.at[i - 1, 'Player']
        r['Next'] = m.at[i + 1, 'Player']
        r['Targ'] = r['Prev'] if r['Prev'] == r['Next'] else np.NaN
        

m

,Code,PlayerLine,Play,Address,Act,Scene,Player,Targ,Prev,Next
0,A Comedy of Errors||||1.1.001||||AEGEON,"Proceed, Solinus, to procure my fallAnd by the...",A Comedy of Errors,1.1.001,1,1,AEGEON,NaN,NaN,DUKE SOLINUS
1,A Comedy of Errors||||1.1.002||||DUKE SOLINUS,"Merchant of Syracuse, plead no more,I am not p...",A Comedy of Errors,1.1.002,1,1,DUKE SOLINUS,AEGEON,AEGEON,AEGEON
2,A Comedy of Errors||||1.1.003||||AEGEON,"Yet this my comfort: when your words are done,...",A Comedy of Errors,1.1.003,1,1,AEGEON,DUKE SOLINUS,DUKE SOLINUS,DUKE SOLINUS
3,A Comedy of Errors||||1.1.004||||DUKE SOLINUS,"Well, Syracusian, say in brief the causeWhy th...",A Comedy of Errors,1.1.004,1,1,DUKE SOLINUS,AEGEON,AEGEON,AEGEON
4,A Comedy of Errors||||1.1.005||||AEGEON,A heavier task could not have been imposedThan...,A Comedy of Errors,1.1.005,1,1,AEGEON,DUKE SOLINUS,DUKE SOLINUS,DUKE SOLINUS
5,A Comedy of Errors||||1.1.006||||DUKE SOLINUS,"Nay, forward, old man, do not break off so,For...",A Comedy of Errors,1.1.006,1,1,DUKE SOLINUS,AEGEON,AEGEON,AEGEON
6,A Comedy of Errors||||1.1.007||||AEGEON,"O, had the gods done so, I had not nowWorthily...",A Comedy of Errors,1.1.007,1,1,AEGEON,DUKE SOLINUS,DUKE SOLINUS,DUKE SOLINUS
7,A Comedy of Errors||||1.1.008||||DUKE SOLINUS,"And for the sake of them thou sorrowest for,Do...",A Comedy of Errors,1.1.008,1,1,DUKE SOLINUS,AEGEON,AEGEON,AEGEON
8,A Comedy of Errors||||1.1.009||||AEGEON,"My youngest boy, and yet my eldest care,At eig...",A Comedy of Errors,1.1.009,1,1,AEGEON,DUKE SOLINUS,DUKE SOLINUS,DUKE SOLINUS
9,A Comedy of Errors||||1.1.010||||DUKE SOLINUS,"Hapless AEgeon, whom the fates have mark'dTo b...",A Comedy of Errors,1.1.010,1,1,DUKE SOLINUS,NaN,AEGEON,Gaoler
